<a href="https://colab.research.google.com/github/ksjeyabarani/W266_Project/blob/master/W266_Project_Model_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We will be creating a table to analyze rouge and other metrics like the one below for varied tensor datasets on base models and on fine-tuned models

In [ ]:
# mount drive to store models and retrieve data
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd '/content/drive/My Drive/W266_Project'

# THis is going to be the directory where we store all resources
ROOT_DIR = '/content/drive/My Drive/W266_Project' 

In [ ]:
# The below command would ensure we are using GPUs
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

  

In [ ]:
# This will check the available RAM for our experiments
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
# Install all necessary packages for our experiments
%cd '/content/drive/My Drive/W266_Project/pegasus'

!export PYTHONPATH=.
!pip3 uninstall tensorflow_estimator
!pip3 install  -r requirements.txt
!sudo apt-get install gcc python-dev python-setuptools libffi-dev
!sudo apt-get install python-pip
!sudo pip install gsutil
!pip3 install apache-beam
!pip3 install pprint

In [ ]:
!pip3 install pandas

import pandas as pd
from collections import defaultdict
from IPython.display import display, HTML

models = ['base', 'cnn', 'multi_news', 'billsum', 'pubmed', 'reddit_tifu', 'aeslc']
testsets = ['cnn', 'multi_news', 'billsum', 'pubmed', 'reddit_tifu', 'aeslc']
bsba = ["bs1_ba5", "bs5_ba5", "bs10_ba5", "bs10_ba1", "bs10_ba9"]
columns = ["model", "testset", "beamsearch_1_α_0.5(precision|recall|f1)",
            "beamsearch_5_α_0.5(precision|recall|f1)",
            "beamsearch_10_α_0.5(precision|recall|f1)",
            "beamsearch_10_α_0.1(precision|recall|f1)",
            "beamsearch_10_α_0.9(precision|recall|f1)"]

data = defaultdict(list)
for m in models:
  for t in testsets:
    data['model'].append(m)
    data['testset'].append(t)
    for v in columns[2:]:
      data[v].append(0)
df = pd.DataFrame(data, columns=columns)
display(df)


,model,testset,beamsearch_1_α_0.5(precision|recall|f1),beamsearch_5_α_0.5(precision|recall|f1),beamsearch_10_α_0.5(precision|recall|f1),beamsearch_10_α_0.1(precision|recall|f1),beamsearch_10_α_0.9(precision|recall|f1)
0,base,cnn,0,0,0,0,0
1,base,multi_news,0,0,0,0,0
2,base,billsum,0,0,0,0,0
3,base,pubmed,0,0,0,0,0
4,base,reddit_tifu,0,0,0,0,0
5,base,aeslc,0,0,0,0,0
6,cnn,cnn,0,0,0,0,0
7,cnn,multi_news,0,0,0,0,0
8,cnn,billsum,0,0,0,0,0
9,cnn,pubmed,0,0,0,0,0


The below steps evaluate each of their datasets with the baseline mode

In [ ]:


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs1_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_cnn_bs10_ba9/.

In [ ]:


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs1_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_aeslc_bs10_ba9/.

In [ ]:


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs1_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_billsum_bs10_ba9/.

In [ ]:


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs1_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_pubmed_bs10_ba9/.

In [ ]:


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs1_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_multi_news_bs10_ba9/.

In [ ]:


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs1_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/model.ckpt-1500000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/inputs-1500000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/targets-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/text_metrics-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/predictions-1500000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/base_reddit_tifu_bs10_ba9/.

The below steps evaluate each of their datasets with their own fine tuned model

In [ ]:
# Evaluating CNN Test dataset (100 records) on finetuned model based on CNN train dataset



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs1_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_bs10_ba9/.

In [ ]:
%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs1_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_bs10_ba9/.

In [ ]:


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs1_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_bs10_ba9/.

In [ ]:


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs1_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba9/.

In [ ]:


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tiku_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tiku/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs1_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tiku_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tiku/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tiku_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tiku/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tiku_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tiku/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tiku_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tiku/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tiku/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tiku_bs10_ba9/.

In [ ]:


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs1_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba9/.

The below steps evaluate other datasets against CNN dailymodel fine tuned model

In [ ]:

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs1_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_aeslc_bs10_ba9/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs1_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs5_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs10_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs10_ba1/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_multi_news_bs10_ba9/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs1_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs5_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_reddit_tifu_bs10_ba9/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs1_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs5_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs10_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs10_ba1/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_billsum_bs10_ba9/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs1_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs5_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs10_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs10_ba1/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail/model.ckpt-210000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/inputs-210000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/targets-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/text_metrics-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/cnn_dailymail/predictions-210000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/cnn_pubmed_bs10_ba9/.



The below steps evaluate other datasets on aeslc fine tuned model

In [ ]:
%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs1_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs5_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs10_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_cnn_bs10_ba9/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs1_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs5_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_multi_news_bs10_ba9/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs1_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs5_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs10_ba1/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_reddit_tifu_bs10_ba9/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs1_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs10_ba1/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_billsum_bs10_ba9/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs1_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs5_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs10_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs10_ba1/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/aeslc/model.ckpt-32000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/inputs-32000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/targets-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/text_metrics-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/aeslc/predictions-32000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/aeslc_pubmed_bs10_ba9/.





The below steps evaluate other datsets on billsum fine tuned model

In [ ]:
%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs1_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs5_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs10_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs10_ba1/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_cnn_bs10_ba9/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs1_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs5_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_aeslc_bs10_ba9/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs1_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs5_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs10_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_multi_news_bs10_ba9/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs1_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs10_ba1/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_reddit_tifu_bs10_ba9/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs1_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs10_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs10_ba1/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/billsum/model.ckpt-180000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/inputs-180000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/targets-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/text_metrics-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/billsum/predictions-180000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/billsum_pubmed_bs10_ba9/.



The below steps generate evaluation for other datasets on reddit trained model

In [ ]:
%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs1_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs5_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_bs10_ba9/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs1_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs5_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs10_ba1/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_cnn_bs10_ba9/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs1_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs5_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs10_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_aeslc_bs10_ba9/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs1_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_multi_news_bs10_ba9/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs1_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs5_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs10_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs10_ba1/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_billsum_bs10_ba9/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs1_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs5_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs10_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu/model.ckpt-8000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/inputs-8000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/targets-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/text_metrics-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/reddit_tifu/predictions-8000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/reddit_tifu_pubmed_bs10_ba9/.



The below steps evaluate other datasets on model fine tuned on pubmed

In [ ]:


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs1_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs5_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba1/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_bs10_ba9/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs1_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs5_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs10_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs10_ba1/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_cnn_bs10_ba9/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs1_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs5_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs10_ba1/.




%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_aeslc_bs10_ba9/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs1_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs5_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs10_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs10_ba1/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_multi_news_bs10_ba9/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs1_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs5_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs10_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs10_ba1/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_reddit_tifu_bs10_ba9/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs1_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs5_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs10_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs10_ba1/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/pubmed/model.ckpt-360000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/inputs-360000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/targets-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/text_metrics-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/pubmed/predictions-360000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/pubmed_billsum_bs10_ba9/.


The below final steps evaluate other datasets on multi_news finetuned model

In [ ]:
%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs1_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba1/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=multi_news_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_bs10_ba9/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs1_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs5_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs10_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs10_ba1/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_cnn_bs10_ba9/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs1_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs5_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs10_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs10_ba1/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_aeslc_bs10_ba9/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs1_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs5_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs10_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_reddit_tifu_bs10_ba9/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs1_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs5_ba5/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs10_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs10_ba1/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=billsum_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_billsum_bs10_ba9/.


%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=1,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs1_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs1_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs1_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=5,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs5_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs5_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs5_ba5/.



%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.5,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs10_ba5
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs10_ba5/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs10_ba5/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.1,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs10_ba1
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs10_ba1/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs10_ba1/.

%cd /content/drive/My Drive/W266_Project/pegasus/
!PYTHONPATH=. python3 pegasus/bin/evaluate.py --params=pubmed_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=2,beam_size=10,beam_alpha=0.9,eval_max_predictions=100 \
--model_dir=ckpt/pegasus_ckpt/multi_news/model.ckpt-60000
!mkdir /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs10_ba9
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/inputs-60000-.test.txt  /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/targets-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/text_metrics-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs10_ba9/.
!cp /content/drive/My\ Drive/W266_Project/pegasus/ckpt/pegasus_ckpt/multi_news/predictions-60000-.test.txt /content/drive/My\ Drive/W266_Project/pegasus/metrics_and_logs/multi_news_pubmed_bs10_ba9/.
